1. Create and Set Up a Stripe Account

Go to Stripe → Sign up.

Verify email, phone, and business/personal details.

Enable Test Mode so you can try payments without real money.

2. Get Your API Keys

In the Dashboard → Developers → API Keys.

Copy:

Publishable Key → for the browser

Secret Key → for the backend (keep it private!)

3. Install Stripe SDK

If you’re using Flask (since I know you prefer Flask from your previous projects):

!pip install stripe flask

4. Backend: Create a Checkout Session

This endpoint will:

Take the donation amount from the frontend

Call Stripe to create a Checkout Session

Send the payment page URL back to the user

In [ ]:
#app.py
import os
import stripe
from flask import Flask, request, jsonify

app = Flask(__name__)

stripe.api_key = "sk_test_your_secret_key_here"

@app.route("/create-checkout-session", methods=["POST"])
def create_checkout_session():
    try:
        amount = int(request.json.get("amount", 0)) * 100  # USD cents

        session = stripe.checkout.Session.create(
            payment_method_types=["card"],
            line_items=[{
                "price_data": {
                    "currency": "usd",
                    "product_data": {
                        "name": "Donation",
                    },
                    "unit_amount": amount,
                },
                "quantity": 1,
            }],
            mode="payment",
            success_url="http://localhost:3000/success",
            cancel_url="http://localhost:3000/cancel",
        )

        return jsonify({"url": session.url})

    except Exception as e:
        return jsonify(error=str(e)), 400

if __name__ == "__main__":
    app.run(port=5000, debug=True)


5. Frontend: Call the Endpoint

Here’s a minimal HTML/JS example for the donate button:

In [ ]:
<button id="donateBtn">Donate $10</button>
<script>
document.getElementById("donateBtn").addEventListener("click", async () => {
    const res = await fetch("/create-checkout-session", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ amount: 10 }) // $10 donation
    });
    const data = await res.json();
    window.location.href = data.url; // redirect to Stripe Checkout
});
</script>


6. Handle Payment Confirmation

Stripe will redirect to your success page when done,
but for automated updates (e.g., sending thank-you emails), you need webhooks:

In [ ]:
@app.route("/webhook", methods=["POST"])
def webhook_received():
    payload = request.data
    sig_header = request.headers.get("Stripe-Signature")
    endpoint_secret = "whsec_your_webhook_secret_here"

    try:
        event = stripe.Webhook.construct_event(
            payload, sig_header, endpoint_secret
        )
    except stripe.error.SignatureVerificationError:
        return "Invalid signature", 400

    if event["type"] == "checkout.session.completed":
        session = event["data"]["object"]
        print("💰 Donation received:", session["amount_total"] / 100, "USD")

    return "", 200


7. Go Live

Switch keys from Test to Live.

Update success/cancel URLs to your real domain.

Make sure HTTPS is enabled.